# PyTorch

In [23]:
# %pip install torch

Note: you may need to restart the kernel to use updated packages.


In [24]:
import torch
x = torch.IntTensor(2, 3)
print(x)

tensor([[1537403456,        440, 1537403440],
        [       440, 1537405376,        440]], dtype=torch.int32)


# Нейронная сеть в PyTorch

Любая нейронная сеть в PyTorch реализуется как класс. У этого класса есть важная особенность: он должен быть унаследован от класса `torch.nn.Module`. 
<br>**Наследование** — это процесс, когда один класс использует атрибуты и методы другого класса как свои собственные, а также расширяет его возможности. Класс, чьи свойства и методы наследуются, называют «суперклассом» или «родителем», а наследующий класс — «подклассом» или «потомком».
## Пример 1
«Точка» — умная колонка. Она умеет изменять громкость от 0 до 10 и обрабатывать запросы пользователей:

In [25]:
class TochkaSmartSpeaker:

    def __init__(self):
        self.volume = 5

    def change_volume(self, volume):
        if volume < 0:
            self.volume = 0
        elif volume > 10:
            self.volume = 10
        else:
            self.volume = volume

    def respond(self, request):
        return 'I am processing your request: ' + request

Разработчики колонки придумали второе поколение «Точки» на основе первого — добавили кнопку отключения звука. Теперь, если звук отключён, то запросы не обрабатываются. Чтобы эта функция заработала, нужно дополнить код. Сначала добавим при инициализации атрибут mute — он будет отвечать за отключение звука:

In [26]:
class TochkaSmartSpeaker2ndGeneration(TochkaSmartSpeaker):

    def __init__(self):
        super().__init__()
        self.is_mute = False

Методом `super()` вызываем соответствующий метод у класса-родителя. В нашем случае, вызывая `super().__init__()`, мы инициализируем поле `volume`. В классе-потомке можно безболезненно создавать дополнительные методы, которые будут включать и выключать звук:

In [27]:
class TochkaSmartSpeaker2ndGeneration(TochkaSmartSpeaker):

    def __init__(self):
        super().__init__()
        self.is_mute = False

    def mute(self):
        self.is_mute = True

    def unmute(self):
        self.is_mute = False

Осталось написать корректную обработку запросов. Когда звук не выключен, колонка второго поколения работает как предыдущая модель. При выключенном звуке в ответ на запрос возвращается пустая строка.

In [28]:
class TochkaSmartSpeaker2ndGeneration(TochkaSmartSpeaker):

    def __init__(self):
        super().__init__()
        self.is_mute = False

    def mute(self):
        self.is_mute = True

    def unmute(self):
        self.is_mute = False

    def respond(self, request):
        return "" if self.is_mute else super().respond(request)

Наследование позволяет использовать неизменное поведение — в примере это регулировка громкости. Если поведение изменилось (появилась кнопка выключения), мы не переписываем весь код, а сохраняем неизменную часть и дописываем то, что поменялось.<br><br>
Вернёмся к задаче построения нейронной сети. В первую очередь нужно создать класс-наследник класса `nn.Module`. Без наследования написать нейронную сеть в PyTorch не получится, вам придётся переписывать очень много различных методов, необходимых для нормального функционирования сети:

In [30]:
import torch.nn as nn

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

Затем нужно определить архитектуру нейронной сети. Данные в `PyTorch` представлены тензорами. 
Полносвязный слой называется `Linear` и находится в библиотеке `torch.nn`. У него есть два параметра: `in_features` — количество нейронов на входе слоя, `out_features` — число нейронов на выходе.
Но в этом полносвязном слое нет функции активации, её нужно применять отдельно. Функции активации находятся в той же библиотеке `torch.nn`. Посмотреть, какие функции активации доступны, можно в документации: https://pytorch.org/docs/stable/nn.html#non-linear-activations-weighted-sum-nonlinearity. 
## Пример 2
Инициализируем полносвязную нейронную сеть из двух полносвязных слоёв. После первого слоя применим сигмоидную функцию активации, а после второго — ReLU.

In [33]:
import torch.nn as nn

class Net(nn.Module):
    def __init__(self, n_in_neurons, n_hidden_neurons, n_out_neurons):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(n_in_neurons, n_hidden_neurons)
        self.act1 = nn.Sigmoid()
        self.fc2 = nn.Linear(n_hidden_neurons, n_out_neurons)        
        self.act2 = nn.ReLU()

Мы объявили архитектуру нейронной сети, но не то, как она будет работать. Теперь необходимо переопределить метод `forward`. Он принимает на вход тензор и возвращает его на выходе, после прохождения через нейронную сеть:

In [34]:
import torch.nn as nn

class Net(nn.Module):
    def __init__(self, n_in_neurons, n_hidden_neurons, n_out_neurons):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(n_in_neurons, n_hidden_neurons)
        self.act1 = nn.Sigmoid()
        self.fc2 = nn.Linear(n_hidden_neurons, n_out_neurons)        
        self.act2 = nn.ReLU()
    def forward(self, x):
        x = self.fc1(x)
        x = self.act1(x)
        x = self.fc2(x)
        x = self.act2(x)
        return x

Осталось объявить объект класса — и можно запускать нейронную сеть:

In [35]:
net = Net(5, 4, 3)

Чтобы прогнать тензор через сеть, нужно вызвать метод `forward`:

In [36]:
output_tensor = net.forward(input_tensor)

NameError: name 'input_tensor' is not defined

## Задача 1
Добавьте в нейронную сеть из примера второй скрытый слой. Для второго скрытого слоя примените гиперболический тангенс `nn.Tanh()` в качестве функции активации.

In [1]:
import torch.nn as nn

class Net(nn.Module):
    def __init__(self, n_in_neurons, n_hidden_neurons_1, n_hidden_neurons_2, n_out_neurons):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(n_in_neurons, n_hidden_neurons_1)
        self.act1 = nn.Sigmoid()
        self.fc2 = nn.Linear(n_hidden_neurons_1, n_hidden_neurons_2) # Ваш код
        self.act2 = nn.Tanh() # Ваш код
        self.fc3 = nn.Linear(n_hidden_neurons_2, n_out_neurons)		
        self.act3 = nn.ReLU()

    def forward(self, x):
        x = self.fc1(x)
        x = self.act1(x)
        x = self.fc2(x)
        x = self.act2(x)# Ваш код
        x = self.fc3(x)
        x = self.act3(x)
        return x

## Задача 2
Инициализируйте нейронную сеть, состоящую из трёх входных нейронов, пяти нейронов в первом скрытом слое, трёх нейронов во втором скрытом слое и одного нейрона в выходном слое. Создайте тензор из значений [1, 44, -7] и прогоните тензор через сеть. Результат выведите на экран.

In [2]:
import torch 
import torch.nn as nn

class Net(nn.Module):
    def __init__(self, n_in_neurons, n_hidden_neurons_1, 
									n_hidden_neurons_2, n_out_neurons):
        super(Net, self).__init__()
			
        self.fc1 = nn.Linear(n_in_neurons, n_hidden_neurons_1)
        self.act1 = nn.Sigmoid()
	
        self.fc2 = nn.Linear(n_hidden_neurons_1, n_hidden_neurons_2)
        self.act2 = nn.Tanh()

        self.fc3 = nn.Linear(n_hidden_neurons_2, n_out_neurons)		
        self.act3 = nn.ReLU()

    def forward(self, x):
        x = self.fc1(x)
        x = self.act1(x)
			
        x = self.fc2(x)
        x = self.act2(x)
			
        x = self.fc3(x)
        x = self.act3(x)
        return x

net = Net(3, 5, 3, 1)  # инициализируйте нейронную сеть 
x = torch.Tensor([1, 44, -7]) # инициализируйте тензор
print(net.forward(x)) # выведите результат прогонки тензора 

tensor([0.], grad_fn=<ReluBackward0>)


## Задача 3
Через нейронную сеть можно прогонять не только отдельные объекты, но и датасеты. Датасеты обрабатываются как тензоры.
Создайте нейронную сеть, содержащую два скрытых слоя: первый содержит `n_hidden_neurons_1`, а второй — `n_hidden_neurons_2`.<br>
После первого скрытого слоя примените `гиперболический тангенс` в качестве функции активации, после второго — `ReLU`, после выходного слоя — `сигмоиду`.<br>
Дополните код прямого распространения в методе `forward()`.<br>
Инициализируйте нейронную сеть, состоящую из `десяти входных нейронов`, с`еми нейронов в первом скрытом слое`, `четырёх нейронов во втором скрытом слое` и `одного нейрона в выходном слое`. Создайте `тензор размером 100 на 10`, состоящий из случайных вещественных значений. Прогоните тензор через сеть. Результат выведите на экран.

In [3]:
import torch.nn as nn

class Net(nn.Module):
    def __init__(self, n_in_neurons, n_hidden_neurons_1, 
									n_hidden_neurons_2, n_out_neurons):
        super(Net, self).__init__()
			
        self.fc1 = nn.Linear(n_in_neurons, n_hidden_neurons_1)
        self.act1 = nn.Tanh()
	
        self.fc2 = nn.Linear(n_hidden_neurons_1, n_hidden_neurons_2)
        self.act2 = nn.ReLU()

        self.fc3 = nn.Linear(n_hidden_neurons_2, n_out_neurons)
        self.act3 = nn.Sigmoid()

    def forward(self, x):
        x = self.fc1(x)
        x = self.act1(x)
			
        x = self.fc2(x)
        x = self.act2(x)
			
        x = self.fc3(x)
        x = self.act3(x)
        return x

net = Net(10, 7, 4, 1) # инициализируйте нейронную сеть 
x = torch.FloatTensor(100, 10) # инициализируйте тензор  
print(net.forward(x)) # выведите результат прогонки тензора 

tensor([[0.6067],
        [0.6191],
        [0.6067],
        [0.6067],
        [0.6260],
        [0.6059],
        [0.6067],
        [0.6067],
        [0.6067],
        [0.6059],
        [0.6067],
        [0.6059],
        [0.6067],
        [0.6059],
        [0.6059],
        [0.6059],
        [0.6115],
        [0.6059],
        [0.6067],
        [0.6067],
        [0.6067],
        [0.6067],
        [0.6067],
        [0.6059],
        [0.6260],
        [0.6069],
        [0.6067],
        [0.6067],
        [0.6067],
        [0.6067],
        [0.6115],
        [0.6069],
        [0.6067],
        [0.6059],
        [0.6067],
        [0.6115],
        [0.6067],
        [0.6067],
        [0.6067],
        [0.6067],
        [0.6115],
        [0.6059],
        [0.6260],
        [0.6067],
        [0.6067],
        [0.6067],
        [0.6067],
        [0.6067],
        [0.6115],
        [0.6055],
        [0.6260],
        [0.6059],
        [0.6067],
        [0.6067],
        [0.6067],
        [0

Этот способ инициализации универсальный, но для простых сетей, таких как полносвязные сети прямого распространения, он выглядит громоздко.
# Сети прямого распространения в PyTorch
Сети прямого распространения создаются альтернативным способом — с помощью контейнера `Sequential`. Этот контейнер строит сеть прямого распространения на основе заданных слоёв. В `Sequential` уже есть метод `forward`, в котором входной тензор последовательно проходит через все слои.<br>
При инициализации объекта класса-контейнера `Sequential` в качестве параметров по очереди передаются слои, которые будут использованы. При этом их порядок изменить уже не получится, нужно будет создавать другой объект.
Инициализируем сеть из прошлого урока, используя контейнер `Sequential`. Так она выглядела при создании через `nn.Module`:

In [4]:
import torch.nn as nn

class Net(nn.Module):
    def __init__(self, n_in_neurons, n_hidden_neurons, n_out_neurons):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(n_in_neurons, n_hidden_neurons)
        self.act1 = nn.Sigmoid()
        self.fc2 = nn.Linear(n_hidden_neurons, n_out_neurons)        
        self.act2 = nn.ReLU()

    def forward(self, x):
        x = self.fc1(x)
        x = self.act1(x)
        x = self.fc2(x)
        x = self.act2(x)
        return x

Так её можно переписать через `Sequential`:

In [5]:
net = nn.Sequential(
    nn.Linear(n_in_neurons, n_hidden_neurons),
    nn.Sigmoid(), 
    nn.Linear(n_hidden_neurons, n_out_neurons), 
    nn.ReLU()
)

NameError: name 'n_in_neurons' is not defined

Здесь мы сразу получаем объект класса-контейнера `Sequential`.<br>
## Задача 1
Перепишите трёхслойную нейронную сеть из задачи 1 прошлого урока: добавьте в нейронную сеть прямого распространения второй скрытый слой. Для второго скрытого слоя примените гиперболический тангенс `nn.Tanh()` в качестве функции активации.
Задайте конкретные значения для количества нейронов: во входном — $4$, в первом скрытом — $6$, во втором скрытом — $3$, в выходном слое — $1$.
Создайте тензор размером $50$ на $4$, состоящий из случайных вещественных значений, и прогоните тензор через сеть.

In [10]:
import torch 
import torch.nn as nn


n_in_neurons = 4# Ваш код
n_hidden_neurons_1 = 6# Ваш код
n_hidden_neurons_2 = 3# Ваш код
n_out_neurons = 1# Ваш код

net = nn.Sequential(
    nn.Linear(n_in_neurons, n_hidden_neurons_1),
    nn.Sigmoid(),
    nn.Linear(n_hidden_neurons_1, n_hidden_neurons_2),
    nn.Tanh(),# Ваш код
    nn.Linear(n_hidden_neurons_2, n_out_neurons), 
    nn.ReLU()
)

x = torch.Tensor(50, 4) # инициализируйте тензор
print(net.forward(x)) # выведите результат прогонки тензора

tensor([[0.0317],
        [0.0317],
        [0.0317],
        [0.0317],
        [0.0317],
        [0.0317],
        [0.0317],
        [0.0317],
        [0.0317],
        [0.0317],
        [0.0317],
        [0.0317],
        [0.0317],
        [0.0317],
        [0.1556],
        [0.0317],
        [0.0317],
        [0.1556],
        [0.1556],
        [0.0317],
        [0.0317],
        [0.0317],
        [0.0317],
        [0.0317],
        [0.0317],
        [0.0317],
        [0.0317],
        [0.0317],
        [0.0317],
        [0.0317],
        [0.1556],
        [0.0317],
        [0.0317],
        [0.0317],
        [0.0317],
        [0.0317],
        [0.0317],
        [0.0317],
        [0.0317],
        [0.1556],
        [0.0317],
        [0.1556],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000]], grad_fn=<ReluBackward0>)


## Задача 2
Создайте полносвязную нейронную сеть с произвольным числом скрытых слоёв. Количество нейронов в каждом слое задано в списке `n_neurons`. Длина списка не меньше $2$. Каждый элемент списка является числом нейронов в соответствующих слоях. В качестве функций активации для нечётных слоёв используйте сигмоиду `nn.Sigmoid()`, для чётных слоёв — гиперболический тангенс `nn.Tanh()`, входной слой считается нулевым. Для выходного слоя используйте функцию активации `nn.ReLU()`. 
Создайте тензор размером $50$ на $20$, состоящий из случайных вещественных значений, и прогоните тензор через сеть.

In [17]:
import torch 
import torch.nn as nn

n_neurons = [20, 16, 12, 8, 4, 2, 1]
net_layers = []

for i in range(1, len(n_neurons) - 1):
		net_layers.append(nn.Linear(n_neurons[i-1], n_neurons[i])) # добавьте полносвязный слой
		if (i+1) % 2 == 0:
				net_layers.append(nn.Tanh()) # добавьте функцию активации для чётных слоёв
		else:
				net_layers.append(nn.Sigmoid()) # добавьте функцию активации для нечётных слоёв

net_layers.append(nn.Linear(n_neurons[-2], n_neurons[-1])) # добавление выходного слоя
net_layers.append(nn.ReLU()) #

net = nn.Sequential(*net_layers) # такая запись позволяет передавать элементы списка как параметры для инициализации
print(net) # вывод архитектуры сети

x = torch.Tensor(50, 20) # инициализируйте тензор
print(net.forward(x)) # выведите результат прогонки тензора

Sequential(
  (0): Linear(in_features=20, out_features=16, bias=True)
  (1): Tanh()
  (2): Linear(in_features=16, out_features=12, bias=True)
  (3): Sigmoid()
  (4): Linear(in_features=12, out_features=8, bias=True)
  (5): Tanh()
  (6): Linear(in_features=8, out_features=4, bias=True)
  (7): Sigmoid()
  (8): Linear(in_features=4, out_features=2, bias=True)
  (9): Tanh()
  (10): Linear(in_features=2, out_features=1, bias=True)
  (11): ReLU()
)
tensor([[0.0290],
        [0.0290],
        [0.0290],
        [0.0290],
        [0.0290],
        [0.0290],
        [0.0290],
        [0.0290],
        [0.0290],
        [0.0290],
        [0.0290],
        [0.0290],
        [0.0290],
        [0.0290],
        [0.0290],
        [0.0290],
        [0.0290],
        [0.0290],
        [0.0290],
        [0.0290],
        [0.0290],
        [0.0290],
        [0.0290],
        [0.0290],
        [0.0290],
        [0.0290],
        [0.0290],
        [0.0290],
        [0.0290],
        [0.0290],
        [0.02

# Функция потерь
$L=L(y, \hat{y})$ на вход этой функции подают результат вычисления сети для объекта $\hat{y}$ и известный ответ $y$. На выходе она выдаст разницу между выводом модели и известным ответом.
1. Метрика качества нужна, чтобы оценить работу модели на новых для неё данных.
2. Функция потерь помогает найти наилучшее значение параметров на тренировочных данных.
<br>Как это было с функциями активации, для разных задач подходят разные функции потерь.

## Задача регрессии
В этой задаче функцией потери может быть среднеквадратичное отклонение (англ. mean squared error, $MSE$) и среднее абсолютное отклонение (англ. mean absolute error, $MAE$). Они выглядят как соответствующие метрики качества:<br>
* $MSE(y, \hat{y})=(y-\hat{y})^2$
* $MAE(y, \hat{y})=|y-\hat{y}|$
<br>В `PyTorch` эти функции потерь находятся в модуле `torch.nn`: `MSELoss`, `L1Loss`.<br>

## Задача бинарной классификации
Вместо функции потерь используют бинарную кросс-энтропию (англ. binary cross-entropy, `BCE`):<br>
$BCE(y, \hat{y})=-y log \hat{y}-(1-y) \log{(1-\hat{y}})$
<br>Чтобы бинарная кросс-энтропия работала корректно, нужно в последнем слое сети использовать `сигмоиду` в качестве функции активации. Тогда значения предсказаний будут от $0$ до $1$.
<br>В `PyTorch` бинарная кросс-энтропия называется `BCELoss` и находится в модуле `torch.nn`.
Если по какой-то причине `сигмоида` или другая функция активации в последнем слое не используется, замените `BCELoss` на `BCEWithLogitsLoss`. В последней функции `сигмоида` применяется перед вычислением кросс-энтропии.

## Задача многоклассовой классификации
Когда нужно определить принадлежность объекта одному из $C$ классов, в роли функции потерь чаще всего применяется обобщение бинарной кросс-энтропии — категориальная кросс-энтропия (англ. categorical cross-entropy, `CCE`):<br>
$CCE(y, \hat{y})=-\sum{_{j=1} ^ {C}} y_i log \hat{y}_j$<br>
$y_j=1$, если объект принадлежит классу $j$, в противном случае $y_j=0$
<br>Для категориальной кросс-энтропии необходимо, чтобы предсказания были от 0 до 1 (как и для бинарной), поэтому в последнем слое нужно применить функцию потерь `Softmax`.<br>
Категориальная кросс-энтропия в `PyTorch` находится в модуле torch.nn и называется `CrossEntropyLoss`.<br>
### Вычислить функции потерь для набора данных можно двумя способами:
1. посчитать среднее арифметическое функции потерь для всех объектов датасета,
2. вычислить суммы значений функции потерь для всех объектов датасета.<br>
За выбор способа отвечает параметр `reduction`. Он может принимать одно из трёх значений: `mean` — для вычисления среднего арифметического (значение по умолчанию), `sum` — для вычисления суммы, `none` — для вычисления вектора из значений функции потерь для конкретных объектов.

## Задача 1
Чтобы решить задачу регрессии с одним скрытым слоем, инициализируйте нейронную сеть прямого распространения, состоящую из пяти входных нейронов, трёх нейронов в скрытом слое и одного нейрона в выходном слое. После каждого слоя добавьте функцию активации `ReLU`.<br>
Создайте тензор `x` из значений [-0.23, -0.2, 0.31, -0.9, 0.2] и прогоните его через сеть. Результат прогонки сохраните в переменной `y_hat`. Создайте тензор y из значения [0.15], в котором будет храниться целевая переменная.
Инициализируйте **функцию потерь** `MAE` и вычислите значение функции потерь для предсказания модели и целевой переменной.


In [18]:
import torch 
import torch.nn as nn


n_in_neurons = 5# число входных нейронов
n_hidden_neurons = 3# число нейронов в скрытом слое 
n_out_neurons = 1# число выходных нейронов 

net = nn.Sequential(
    nn.Linear(n_in_neurons, n_hidden_neurons)
    , nn.ReLU()
    , nn.Linear(n_hidden_neurons, n_out_neurons)
    , nn.ReLU()
)
print(0)
x = torch.Tensor([-0.23, -0.2, 0.31, -0.9, 0.2])
print(1)
y = torch.Tensor([0.15])
print(2)
y_hat = net.forward(x)# прогоните x через нейронную сеть
print(3)

loss = nn.L1Loss()# инициализируйте функцию потерь MAE
print(4)

print(loss(y_hat, y))

0
1
2
3
4
tensor(0.1402, grad_fn=<MeanBackward0>)


## Задача 2
Измените сеть из предыдущей задачи так, чтобы она могла решить задачу `бинарной классификации`. Для этого после выходного слоя замените функцию активации на `сигмоиду`.<br>
Создайте тензор x из значений [-0.23, -0.2, 0.31, -0.9, 0.2] и прогоните тензор через сеть. Результат сохраните в переменной `y_hat`. Создайте тензор y из значения [0], в котором будет храниться целевая переменная.<br>
Инициализируйте функцию потерь `бинарной кросс-энтропии` и вычислите значение функции потерь для предсказания модели и целевой переменной.<br>
Модель на выходе выдаёт вещественное число от 0 до 1 — «вероятность» принадлежности классу 1. Чтобы перевести это число в класс 0 или 1, сравните его с порогом 0.5: если выход модели больше 0.5, должен быть класс 1, в противном случае — 0.

In [19]:
import torch 
import torch.nn as nn


n_in_neurons = 5 
n_hidden_neurons = 3
n_out_neurons = 1 

net = nn.Sequential(
    nn.Linear(n_in_neurons, n_hidden_neurons),
    nn.ReLU(),
    nn.Linear(n_hidden_neurons, n_out_neurons), 
    nn.Sigmoid()
)

x = torch.Tensor([-0.23, -0.2, 0.31, -0.9, 0.2])
y = torch.Tensor([0])
y_hat = net.forward(x)# прогоните x через нейронную сеть

loss = nn.BCELoss()# инициализируйте функцию потерь бинарной кросс-энтропии

print(loss(y_hat, y))

print(int(y_hat > 0.5 )) # выведите класс, которому принадлежит данный объект, по мнению модели

tensor(0.5508, grad_fn=<BinaryCrossEntropyBackward0>)
0


## Задача 3
Измените сеть из предыдущей задачи так, чтобы она решала задачу `многоклассовой классификации`. Для этого замените число нейронов на выходе из сети на $3$ и после выходного слоя замените функцию активации на `Softmax()`.<br>
Создайте тензор x из значений [[-0.23, -0.2, 0.31, -0.9, 0.2], [0.23, 0.2, -0.31, 0.9, -0.2]] и прогоните тензор через сеть. Результат сохраните в переменной `y_hat`. Создайте тензор y из значения [2, 0], в котором будет храниться целевая переменная.<br>
Инициализируйте функцию потерь кросс-энтропии и вычислите значение функции потерь для предсказания модели и целевой переменной.<br>
Модель на выходе выдаёт «вероятность» принадлежности каждому из классов. Выведите класс методом `argmax(dim=1)`, который для каждого предсказания даёт индекс максимального элемента списка вероятностей — наиболее вероятный класс.

In [20]:
import torch 
import torch.nn as nn


n_in_neurons = 5 
n_hidden_neurons = 3
n_out_neurons = 3 

net = nn.Sequential(
    nn.Linear(n_in_neurons, n_hidden_neurons),
    nn.ReLU(),
    nn.Linear(n_hidden_neurons, n_out_neurons), 
    nn.Softmax()
)

x = torch.Tensor([[-0.23, -0.2, 0.31, -0.9, 0.2], [0.23, 0.2, -0.31, 0.9, -0.2]])

y = torch.LongTensor([2, 0])

y_hat = net.forward(x) # прогоните x через нейронную сеть

loss = nn.CrossEntropyLoss()# инициализируйте функцию потерь категориальной кросс-энтропии

print(loss(y_hat, y))

print(y_hat)
print(torch.argmax(y_hat, dim=1) ) # выведите класс, которому принадлежит данный объект, по мнению модели

D:\Programs\lib\site-packages\torch\nn\modules\container.py:204: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


tensor(1.1406, grad_fn=<NllLossBackward0>)
tensor([[0.3439, 0.3081, 0.3481],
        [0.2370, 0.3807, 0.3823]], grad_fn=<SoftmaxBackward0>)
tensor([2, 2])


# Обратное распространение ошибки
Градиент показывает направление наискорейшего возрастания функции, антиградиент (отрицательный градиент) — направление наибольшего убывания, формулa для градиентного спуска:<br>
$\omega^{t+1}=\omega^t+\Delta{\omega^t}$<br>
$\Delta{\omega^t}=-\eta\frac{dL}{d\omega^t}$<br>
Где $\omega^t$, $\omega^{t+1}$

НУЖНО ДОПИСАТЬ ПРО ГРАДИЕНТЫ, ФУНКЦИИ ПОТРЕЬ И Т.П.


# Тренировка модели
Чтобы использовать оптимизатор в обучении модели, сначала его нужно объявить. В первую очередь в оптимизатор передают значения, которые нужно оптимизировать. Если это нейронная сеть, то нужно передать её параметры с помощью метода `parameters()`. Затем можно указать параметры алгоритма оптимизации. Например, скорость обучения — `learning rate`, `lr`:

In [2]:
net = Net()
...
optimizer = torch.optim.Adam(net.parameters(), lr=1.0e-3)

NameError: name 'Net' is not defined

Скорость обучения можно менять. <i>Возьмёте слишком большую — рискуете «проскочить» минимум, слишком маленькую — алгоритм будет работать очень медленно.</i><br>
В `PyTorch` оптимизаторы имеют свойство накапливать градиент. Это может повредить обучению, поэтому перед каждым шагом нужно обнулить накопленный градиент методом `zero_grad()`. Затем вычисляются текущие предсказания модели, подсчитывается значение функции потерь. После этого с помощью метода `backward()` вычисляется значение градиента функции потерь и выполняется шаг работы оптимизатора (обновление весов в сети):

In [3]:
optimizer.zero_grad()

preds = net.forward(X) 
        
loss_value = loss(preds, y)
loss_value.backward()
        
optimizer.step()

NameError: name 'optimizer' is not defined

Еще раз последовательность должна быть такая:
1. Сделать шаг оптимизации, вызвав метод `step()`.
2. Получить предсказания с текущими весами, прогнав признаки через сеть с помощью `forward()`.
3. Получить значение функции потерь для предсказаний и истинных значений.
4. Вычислить градиенты функции потерь с помощью метода `backward()`.
5. Сделать шаг оптимизации, вызвав метод `step()`.
# Обучение нейронной сети в PyTorch
## Эпоха обучения
Нейронные сети обучаются в несколько итераций — эпох. В одной эпохе происходит обнуление градиентов в оптимизаторе, прямое распространение, подсчёт значения функции потерь, вычисление градиента для функции потерь, изменение весов с помощью оптимизатора, вычисление метрики качества. <u>Перед вычислением метрики качества желательно перевести модель в режим предсказания — вызвать метод `eval()`</u>. В этом режиме не будут вычисляться и накапливаться градиенты. Перед вами фрагмент кода обучения сети для одной эпохи:

In [ ]:
optimizer.zero_grad() # обнуление градиентов

preds = net.forward(X_train)  # прямое распространение на обучающих данных
        
loss_value = loss(preds, y_train) # вычисление значения функции потерь
loss_value.backward() # вычисление градиентов
        
optimizer.step() # один шаг оптимизации весов

net.eval() # перевод сети в режим предсказания
test_preds = net.forward(X_test) # прямое распространение на тестовых данных
accuracy = (test_preds.argmax(dim=1) == y_test).float().mean().data # вычисление доли правильных ответов

Число эпох задаётся перед обучением, оно может быть любым. Такой фрагмент кода обучает сеть за $100$ эпох: 

In [4]:
num_epochs = 100

for epoch in range(num_epochs):
    optimizer.zero_grad()
    
    preds = net.forward(X_train) 
            
    loss_value = loss(preds, y_train)
    loss_value.backward()
            
    optimizer.step()
    
    net.eval()
    test_preds = net.forward(X_test)
    accuracy = (test_preds.argmax(dim=1) == y_test).float().mean().data
    
    print(accuracy)

NameError: name 'optimizer' is not defined

Иногда метрику качества вычисляют не каждую эпоху, а, например, каждую пятую или десятую:

In [ ]:
num_epochs = 100

for epoch in range(num_epochs):
    optimizer.zero_grad()
    
    preds = net.forward(X_train) 
            
    loss_value = loss(preds, y_train)
    loss_value.backward()
            
    optimizer.step()
    
    if epoch % 10 == 0:
        net.eval()
        test_preds = net.forward(X_test)
        accuracy = (test_preds.argmax(dim=1) == y_test).float().mean().data
        
        print(accuracy)

## Батчи
Когда идёт обучение на больших датасетах, не всегда получается загрузить все нужные данные в оперативную память. Поэтому в каждой эпохе нейронная сеть получает данные частями — батчами (`batch`).
Как получать батчи — сейчас расскажем. В начале эпохи нужно сгенерировать случайную перестановку объектов обучающей выборки (а точнее, их индексов). Это нужно, чтобы батчи изменялись от эпохи к эпохе, иначе тренировка будет неэффективной. Затем нужно получить индекс текущего батча и сформировать подвыборку из обучающей выборки.

In [5]:
from math import ceil

batch_size = 100

num_epochs = 1000

num_batches = ceil(len(X_train)/batch_size)

for epoch in range(num_epochs):
    # случайная перестановка объектов
    order = np.random.permutation(len(X_train))
    for batch_idx in range(num_batches):
        start_index = batch_idx * batch_size
        optimizer.zero_grad()
        
        # получение индексов текущего батча
        batch_indexes = order[start_index:start_index+batch_size]
        X_batch = X_train[batch_indexes]
        y_batch = y_train[batch_indexes]
    
        preds = net.forward(X_batch) 
                
        loss_value = loss(preds, y_batch)
        loss_value.backward()
                
        optimizer.step()
    
    if epoch % 10 == 0:
        net.eval()
        test_preds = net.forward(X_test)
        accuracy = (test_preds.argmax(dim=1) == y_test).float().mean().data
        
        print(accuracy)

NameError: name 'X_train' is not defined

Один шаг обучения сети на конкретном батче называется итерацией (`iteration`). В рамках одной эпохи обучение на батчах производится в несколько итераций.<br>
Эффект накопления градиентов в `PyTorch` работает так: старые градиенты весов и сдвигов нейронной сети после обратного распространения ошибки не заменяются на новые, а суммируются с ними. Эту особенность `PyTorch` используют для ускорения обучения: можно делать шаг алгоритма оптимизации не для каждого батча, а для нескольких батчей, например $5$ или $10$. <i>Но если функция потерь для батча вычисляется как среднее арифметическое по всем его элементам (поведение по умолчанию), нужно усреднить значение этой функции</i>. Так выглядит обучение с накоплением градиентов:

In [ ]:
from math import ceil

batch_size = 100

num_epochs = 1000

accumulation_iteration = 5 # делать оптимизационный шаг каждый 5-й батч

num_batches = ceil(len(X_train)/batch_size)

for epoch in range(num_epochs):
    # случайная перестановка объектов
    order = np.random.permutation(len(X_train))
    optimizer.zero_grad()
    for batch_i in range(num_batches):
        start_index = batch_i * batch_size
        
        # получение индексов текущего батча
        batch_indexes = order[start_index:start_index+batch_size]
        X_batch = X_train[batch_indexes]
        y_batch = y_train[batch_indexes
    
        preds = net.forward(X_batch) 
                
        loss_value = loss(preds, y_batch) / accumulation_iteration
        loss_value.backward()
                
        if ((batch_i + 1) % accumulation_iteration == 0) or (batch_i + 1 == num_batches):
            optimizer.step()
            optimizer.zero_grad()
    
    if epoch % 10 == 0:
        net.eval()
        test_preds = net.forward(X_test)
        accuracy = (test_preds.argmax(dim=1) == y_test).float().mean().data
        
        print(accuracy)

### <font color = 'blue'>Задача обучения нейронной сети</font>
Спрогнозируйте стабильность электрических сетей. Вам предоставили файл с данными:
 '/datasets/Electrical_Grid_Stability.csv'.
Признаки обезличены: по названиям вы никак не сможете понять, что они значат. Целевая переменная `stability` отвечает за стабильность сети: если её значение — '1', то сеть стабильна, '0' — нет.
Начните работу над задачей: подготовьте данные и инициализируйте нейронную сеть.
С помощью метода `train_test_split` разделите данные на обучающие и тестовые. Тестовые данные должны составлять $0.3$ от размера датасета, используйте параметр `test_size`. Укажите параметр `shuffle=True` для того, чтобы перемешать датасет перед разделением. Для воспроизведения результатов укажите параметр `random_state=42`.<br>
Инициализируйте нейронную сеть прямого распространения с двумя скрытыми слоями: 
<br>$12$ входных нейронов, 
<br>$8$ нейронов в первом скрытом слое, 
<br>$4$ нейрона во втором скрытом слое и один нейрон в выходном слое.
<br>После первого скрытого слоя примените функцию активации `ReLU`, после второго — `гиперболический тангенс`, после выходного — `сигмоиду`.
<br>Постройте цикл обучения нейронной сети, которая будет прогнозировать стабильность электрических цепей.
Инициализируйте оптимизатор `Adam` с параметром шага $lr=1e-3$.
<br>Объявите функцию потерь бинарной кросс-энтропии.
<br>Дополните цикл обучения сети. Каждую десятую эпоху и в последнюю эпоху обучения проверяйте качество на тестовых данных. <br>В качестве метрики качества используйте долю правильных ответов.
<br>

In [ ]:
import random
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from math import ceil

from sklearn.model_selection import train_test_split


random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
torch.use_deterministic_algorithms(True)

data = pd.read_csv('/datasets/Electrical_Grid_Stability.csv', sep=';')

X_train, X_test, y_train, y_test = train_test_split(
    data.drop(columns=['stability']), 
    data.stability, 
    test_size=0.3, 
    shuffle=True)

X_train = torch.FloatTensor(X_train.values)
X_test = torch.FloatTensor(X_test.values)
y_train = torch.FloatTensor(y_train.values)
y_test = torch.FloatTensor(y_test.values)


n_in_neurons = 12
n_hidden_neurons_1 = 8
n_hidden_neurons_2 = 4
n_out_neurons = 1 

net = nn.Sequential(
    nn.Linear(n_in_neurons, n_hidden_neurons_1),
    nn.ReLU(),
    nn.Linear(n_hidden_neurons_1, n_hidden_neurons_2),
    nn.Tanh(),
    nn.Linear(n_hidden_neurons_2, n_out_neurons), 
    nn.Sigmoid()
)


optimizer = torch.optim.Adam(net.parameters(), lr=1e-3)

loss = nn.BCELoss()

batch_size = 500

num_epochs = 100

num_batches = ceil(len(X_train)/batch_size)

for epoch in range(num_epochs):
	order = np.random.permutation(len(X_train)) # создайте случайную перестановку индексов объектов
	for batch_idx in range(num_batches):
		start_index = batch_idx * batch_size# посчитайте номер стартового объекта батча
		optimizer.zero_grad()
  
		batch_indexes = order[start_index:start_index+batch_size] # извлеките индексы объектов текущего обатча
		X_batch = X_train[batch_indexes]
		y_batch = y_train[batch_indexes]
  
		preds = net.forward(X_batch).flatten()
	        
		loss_value = loss(preds, y_batch)

		loss_value.backward()
	        
		optimizer.step()
		
	if epoch % 10 == 0 or epoch == num_epochs - 1:
		net.eval()
		test_preds = net.forward(X_test)
		accuracy = (torch.round(test_preds) == y_test).float().mean().data
		print(accuracy)


Вывод
<br>tensor(0.6463)
<br>tensor(0.6081)
<br>tensor(0.5661)
<br>tensor(0.5615)
<br>tensor(0.5467)
<br>tensor(0.5480)
<br>tensor(0.5419)
<br>tensor(0.5455)
<br>tensor(0.5404)
<br>tensor(0.5503)
<br>tensor(0.5417)

# Инициализация параметров сети
От того, как вы инициализируете сеть, зависит, как быстро функция потерь придёт к оптимальному значению.<br>
Нейронная сеть в `PyTorch` создаётся либо как наследник `nn.Module`, либо с помощью `Sequential`
## Инициализация равномерным распределением `nn.Module`
В PyTorch методы инициализации весов находятся в модуле `torch.nn.init`. Если выбрана инициализация равномерным распределением, нужно использовать метод `uniform_()` и применить его ко всем параметрам, указав границы распределения `a` и `b` — $0$ и $1$ по умолчанию. Для доступа к параметрам полносвязного слоя используйте атрибуты `.weigth` и `.bias` — веса и отступы:


In [ ]:
import torch.nn as nn

class Net(nn.Module):
    def __init__(self, n_in_neurons, n_hidden_neurons, n_out_neurons):
      super(Net, self).__init__()
            
      self.fc1 = nn.Linear(n_in_neurons, n_hidden_neurons)
      self.act1 = nn.Sigmoid()
      self.fc2 = nn.Linear(n_hidden_neurons, n_out_neurons)        
      self.act2 = nn.ReLU()
            
      nn.init.uniform_(self.fc1.weight, a=-1, b=2)
      nn.init.uniform_(self.fc1.bias, a=1, b=2)
      nn.init.uniform_(self.fc2.weight, b=3)
      nn.init.uniform_(self.fc2.bias, a=-1)
   

    def forward(self, x):
      x = self.fc1(x)
      x = self.act1(x)
      x = self.fc2(x)
      x = self.act2(x)
      return x

## Инициализация нормальным распределением `Sequential`
Теперь выберем сеть, созданную с помощью `Sequential`, и инициализируем её нормальным распределением. Для этого распределения используется метод `normal_`. Параметры метода: `mean` — медиана и `std` — стандартное отклонение. 
Сначала напишите метод, который будет инициализировать слой:

In [ ]:
def init_weights(layer):
    if type(layer) == nn.Linear: # Проверка, что слой — полносвязный
        nn.init.normal_(layer.weight, mean=0.5, std=0.5)
        nn.init.normal_(layer.bias, mean=-0.5, std=1.5)

Затем, примените его ко всем слоям сети с помощью метода `apply`:

In [ ]:
net = nn.Sequential(
    nn.Linear(n_in_neurons, n_hidden_neurons),
    nn.Sigmoid(), 
    nn.Linear(n_hidden_neurons, n_out_neurons), 
    nn.ReLU()
)

net.apply(init_weights)

Осталось пояснить, как выбирать параметры распределений для инициализации. Этот выбор зависит от того, насколько функция активации сети симметрична относительно $0$.
## Выбор параметров при симметричной функции активации
способ называют в честь одного из авторов статьи инициализацией Ксавье (Xavier initialization) или инициализацией Глоро (Glorot initialization). Чтобы использовать этот метод в PyTorch, вызовите метод `xavier_uniform_` или `xavier_normal_`. Инициализацию Ксавье в PyTorch можно применять только к весам.

In [ ]:
def init_weights(layer):
    if type(layer) == nn.Linear: # Проверка, что слой – полносвязный
        nn.init.xavier_uniform_(layer.weight)

## Выбор параметров в случае, если функция активации несимметрична
Это называется инициализацией Кайминга (Kaiming initialization) или инициализацией Хе (He initialization). Чтобы использовать этот метод в PyTorch, вызовите метод `kaiming_uniform_` или `kaiming_normal_`. У обоих методов есть параметры, которые нужно использовать. Во-первых, нужно указать функцию активации `ReLU`: `nonlinearity='relu'`. Во-вторых, нужно указать в параметре `mode`, в каком случае нужно, чтобы дисперсия весов сохранялась: при прямом `('fan_in')` или при обратном `('fan_out')` распространении. Инициализацию Кайминга в PyTorch можно применять только к весам.

In [ ]:
def init_weights(layer):
    if type(layer) == nn.Linear: # Проверка, что слой – полносвязный
        nn.init.kaiming_uniform_(layer.weight, mode='fan_in', nonlinearity='relu')

По умолчанию в PyTorch веса и смещения инициализируются случайными числами из равномерного распределения с границами<br>
$$
a = - \sqrt{\frac{1}{n_{i-1}}}, b = \sqrt{\frac{1}{n_{i-1}}}
$$
Удачная инициализация весов помогает быстрее найти минимум функции, а в некоторых случаях предотвращает попадание в неподходящий локальный минимум и повышает качество модели.
Инициализация весов — это не единственный способ улучшить нейронную сеть.
### Задача 1

Инициализируйте нейронную сеть прямого распространения с двумя скрытыми слоями слоем, состоящую из 12 входных нейронов, 8 нейронов в первом скрытом слое, 4  нейронов во втором скрытом слое и <u>одного</u> нейрона в выходном слое.
<br>После первого скрытого слоя примените функцию активации `гиперболический тангенс`, после второго скрытого — `ReLU`, после выходного слоя — `сигмоиду`.
Инициализируйте веса и смещения:
<br>В первом слое равномерным распределением от −2 до 2.
<br>Во втором слое веса — с помощью инициализации Кайминга, смещения — нормальным распределением с математическим ожиданием 0.5 и среднеквадратичным отклонением 0.7.
<br>В выходном слое веса — с помощью нормальной инициализации `Ксавье`, смещения — `нормальным распределением с математическим ожиданием ` 0.5 и `среднеквадратичным отклонением` 0.7.
<br>Инициализируйте оптимизатор `Adam` с параметром шага $lr=1e-3$.
<br>Объявите функцию потерь `бинарной кросс-энтропии`.
<br>Дополните цикл обучения сети. Каждую 10-ю эпоху и в последнюю эпоху обучения проверяйте качество на тестовых данных. В качестве метрики качества используйте долю правильных ответов.

In [ ]:
import random
import numpy as np
import pandas as pd
import torch
import torch.nn as nn

from sklearn.model_selection import train_test_split

random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.use_deterministic_algorithms(True)


class Net(nn.Module):
    def __init__(self, n_in_neurons, n_hidden_neurons_1,
                 n_hidden_neurons_2, n_out_neurons):
        super(Net, self).__init__()

        self.fc1 = nn.Linear(n_in_neurons, n_hidden_neurons_1)
        self.act1 = nn.Tanh()

        self.fc2 = nn.Linear(n_hidden_neurons_1, n_hidden_neurons_2)
        self.act2 = nn.ReLU()

        self.fc3 = nn.Linear(n_hidden_neurons_2, n_out_neurons)
        self.act3 = nn.Sigmoid()
        
        
        

        nn.init.uniform_(self.fc1.weight, a=-2, b=2) #равномерным распределением от −2 до 2.
        nn.init.uniform_(self.fc1.bias, a=-2, b=2)
        nn.init.kaiming_normal_(self.fc2.weight, mode='fan_in', nonlinearity='relu') #с помощью инициализации Кайминга
        nn.init.normal_(self.fc2.bias, mean=0.5, std=0.7) #с помощью инициализации Кайминга
        nn.init.xavier_normal_(self.fc3.weight)
        nn.init.normal_(self.fc3.bias, mean=0.5, std=0.7)

    def forward(self, x):
        x = self.fc1(x)
        x = self.act1(x)

        x = self.fc2(x)
        x = self.act2(x)

        x = self.fc3(x)
        x = self.act3(x)
        return x


data = pd.read_csv('/datasets/Electrical_Grid_Stability.csv', sep=';')

X_train, X_test, y_train, y_test = train_test_split(
    data.drop(columns=['stability']),
    data.stability,
    test_size=0.3,
    shuffle=True)

X_train = torch.FloatTensor(X_train.values)
X_test = torch.FloatTensor(X_test.values)
y_train = torch.FloatTensor(y_train.values)
y_test = torch.FloatTensor(y_test.values)


n_in_neurons = 12
n_hidden_neurons_1 = 8
n_hidden_neurons_2 = 4
n_out_neurons = 1

net = Net(n_in_neurons
          , n_hidden_neurons_1
          , n_hidden_neurons_2
          , n_out_neurons) #nn.Sequential


optimizer = torch.optim.Adam(net.parameters(), lr=1e-3) ###&&&&

loss = nn.BCELoss()

num_epochs = 100

for epoch in range(num_epochs):
    optimizer.zero_grad()
    order = np.random.permutation(len(X_train)) # создайте случайную перестановку индексов объектов
    preds = net.forward(X_train).flatten()
    loss_value = loss(preds, y_train)
    loss_value.backward()
    optimizer.step()
    if epoch % 10 == 0 or epoch == num_epochs - 1:
        net.eval()
        test_preds = net.forward(X_test)
        accuracy = (test_preds.argmax(dim=1) == y_test).float().mean().data
        print(accuracy)

**<u><font color = 'blue'>Создайте полносвязную нейронную сеть с произвольным числом скрытых слоёв</font></u>**. Количество нейронов в каждом слое задано в списке `n_neurons`. Длина списка не меньше двух. Каждый элемент списка является числом нейронов в соответствующих слоях. В качестве функции активации для нечётных слоёв используйте `nn.ReLU()`, для чётных — гиперболический тангенс `nn.Tanh()`, входной слой считается нулевым. Для выходного слоя используйте функцию активации `nn.Sigmoid()`. 
<br>Создайте метод `init_weights` для инициализации полносвязных слоёв. Инициализируйте веса с помощью нормального распределения с математическим ожиданием 0.5 и среднеквадратичным отклонением 2, а смещения — с помощью нормального −0.5 и среднеквадратичным отклонением 1.

In [ ]:
import random
import numpy as np
import pandas as pd
import torch
import torch.nn as nn

from sklearn.model_selection import train_test_split


random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.use_deterministic_algorithms(True)

data = pd.read_csv('/datasets/Electrical_Grid_Stability.csv', sep=';')

X_train, X_test, y_train, y_test = train_test_split(
    data.drop(columns=['stability']),
    data.stability,
    test_size=0.3,
    shuffle=True)

X_train = torch.FloatTensor(X_train.values)
X_test = torch.FloatTensor(X_test.values)
y_train = torch.FloatTensor(y_train.values)
y_test = torch.FloatTensor(y_test.values)

n_neurons = [12, 9, 6, 3, 1]
net_layers = []

for i in range(1, len(n_neurons) - 1):
    #net_layers.append(...)
    net_layers.append(nn.Linear(n_neurons[i-1], n_neurons[i]))
    if (i+1) % 2 == 0:
        net_layers.append(nn.Tanh())
    else:
        net_layers.append(nn.ReLU())

net_layers.append(nn.Linear(n_neurons[-2], n_neurons[-1]))
net_layers.append(nn.Sigmoid())

net = nn.Sequential(*net_layers) # такая запись позволяет передавать элементы списка как параметры для инициализации


def init_weights(layer):
    if type(layer) == nn.Linear: # Проверка, что слой – полносвязный
        nn.init.normal_(layer.weight, mean=0.5, std=2)
        nn.init.normal_(layer.bias, mean=-0.5, std=1)

net.apply(init_weights)

optimizer = torch.optim.Adam(net.parameters(), lr=1e-3)

loss = nn.BCELoss()

num_epochs = 100

for epoch in range(num_epochs):
    optimizer.zero_grad()

    preds = net.forward(X_train).flatten()

    loss_value = loss(preds, y_train)
    # print(loss_value)

    loss_value.backward()

    optimizer.step()

    if epoch % 10 == 0 or epoch == num_epochs - 1:
        net.eval()
        test_preds = net.forward(X_test)
        accuracy = (torch.round(test_preds) == y_test).float().mean().data
        print(accuracy)

# Регуляризация весов
Регуляризация весов — это подход, при котором модель штрафуется за большие значение весов. Есть два основных способа начислять штраф: `L1` и `L2` регуляризация. Числа означают степень, в которую будут возводить веса. **`L1` — взятие модуля, а `L2` — возведение в квадрат**.<br>
Pегуляризация предотвращает переобучение, ограничивая значения весов, a также помогает решать проблему взрыва градиента<br>
Регуляризация весов — дополнительное слагаемое к функции потерь. Вместе с ним общая функция потерь Loss имеет вид:
$$
Loss = Error(y, \hat{y})+λ*Loss_{reg}
$$
<br>Где:
 <br>$Error(y, \hat{y})$ - обычная функция потерь (например, кросс-энтропия для классификации);
 <br>$λ$-весовой коэффициент, или лямбда, со значением порядка одной сотой или тысячной. Лямбда — это гиперпараметр, который подбирается для конкретной задачи;
<br>$Loss_{reg}$— регуляризационная составляющая.

## `L1` регуляризация
`L1` регуляризацию легко добавить на PyTorch. Пусть у вас есть модель `model` и  функция потерь `loss`:
<br>

In [ ]:
...
l1_lambda = 0.001
l1_norm = sum(p.abs().sum() for p in model.parameters())
result_loss = loss + l1_lambda * l1_norm
...

## `L2` регуляризация
`L2` регуляризация — это частный случай регуляризации по Тихонову. Она очень похожа на `L1`

In [ ]:
...
l2_lambda = 0.001
l2_norm = sum(p.pow(2.0).sum() for p in model.parameters())
result_loss = loss + l2_lambda * l2_norm
...

Кроме того, для регуляризации можно одновременно использовать и `L1` и `L2`:

In [ ]:
...
l1_lambda = 0.0005
l1_norm = sum(p.abs().sum() for p in model.parameters())
l2_lambda = 0.001
l2_norm = sum(p.pow(2.0).sum() for p in model.parameters())
result_loss = loss + l1_lambda * l1_norm + l2_lambda * l2_norm
...

Значения лямбды не всегда одинаковы. Вам предстоит подбирать их под условия каждой задачи.<br>
## Применимость регуляризации весов
До недавнего времени регуляризация весов применялась повсеместно и была почти обязательной. Сейчас же она нужна крайне редко, в том числе из-за `Batch Normalization`. Кроме того, регуляризация весов если и увеличивает метрики, то незначительно.

## Batch Normalization
Batch Normalization, или BatchNorm, — это метод регуляризации и стабилизации обучения.

In [ ]:
class Model(nn.Module):
    def __init__(self, input_dim):
        super(Model, self).__init__()
        self.layer1 = nn.Linear(input_dim, 50)
        self.layer2 = nn.Linear(50, 200)
        self.layer3 = nn.Linear(200, 3)
        
    def forward(self, x):
        x = self.layer1(x)
        x = torch.relu(x)
        x = self.layer2(x)
        x = torch.relu(x)
        x = self.layer3(x)
        
        x = F.softmax(x, dim=1)
        return x

Теперь добавим к ней BatchNorm:

In [ ]:
class Model(nn.Module):
    def __init__(self, input_dim):
        super(Model, self).__init__()
        self.layer1 = nn.Linear(input_dim, 50)
        self.bn1 = nn.BatchNorm1d(50)
        self.layer2 = nn.Linear(50, 200)
        self.bn2 = nn.BatchNorm1d(200)
        self.layer3 = nn.Linear(200, 3)
        
    def forward(self, x):
        x = self.layer1(x)
        x = self.bn1(x)
        x = torch.relu(x)
        x = self.layer2(x)
        x = self.bn2(x)
        x = torch.relu(x)
        x = self.layer3(x)
        
        x = F.softmax(x, dim=1)
        return x

До сих пор нет консенсуса о том, куда именно лучше добавить Batch Normalization: до или после активации<br>
Чаще всего нормализация работает в обоих случаях.<br>
Batch Normalization — это распространённый и эффективный метод регуляризации и стабилизации обучения с помощью нормализации батча. В популярных фреймворках его легко добавить в виде дополнительного слоя сети.
## Dropout
Она основана на очень простой, но рабочей идее: «выключить» часть нейронов сети.<br>
Единственное, что можно настроить при использовании Dropout, — это доля «выключенных» нейронов $p$. Если применить Dropout c гиперпараметром $p=0.25$ к слою из 128 нейронов, то перестанет работать четверть нейронов: 32 (128/4). «Включённым» или «выключенным» может быть только целый нейрон. Если при делении числа нейронов на $p$ получается дробь, то она округлится по принятым во фреймворке правилам.<br>
У нейрона нет рубильника. Чтобы его «выключить», нужно поменять значения выходов нейрона на нулевые. Для этого их нужно умножить на матрицу из нулей и единиц. Число нулей равняется количеству «выключенных», в котором случайные $p$ элементов нули, а остальные — единицы.<br>
При использовании Dropout нейроны не «выключаются» раз и навсегда. На каждом шаге обучения не работают случайные нейроны: любой из них на текущей итерации может иметь значение ноль, а на следующей принять своё исходное.<br>
Пример сети с Dropout:

In [ ]:
class Model(nn.Module):
    def __init__(self, input_dim):
        super(Model, self).__init__()
        self.layer1 = nn.Linear(input_dim, 50)
        self.dp1 = nn.Dropout(p=0.2)
        self.layer2 = nn.Linear(50, 200)
        self.dp2 = nn.Dropout(p=0.5)
        self.layer3 = nn.Linear(200, 3)
        
    def forward(self, x):
        x = self.layer1(x)
        x = torch.relu(x)
        x = self.dp1(x)
        x = self.layer2(x)
        x = torch.relu(x)
        x = self.dp2(x)
        x = self.layer3(x)
        
        x = F.softmax(x, dim=1)
        return x

Количество слоёв Dropout и их значения $p$ — это гиперпараметры, которые можно подгонять под задачу. Кроме того, Dropout можно применять к любому тензору без контекста обучения. Например:

In [ ]:
m = nn.Dropout(p=0.5)
input_ = torch.randn(2, 10)
print(m(input_))

Обратите внимание, что всего было обнулено десять значений из двадцати, но не по пять в каждой строке, а четыре в первой и шесть во второй. То есть вероятность обнуления применяется ко всему тензору, а не к отдельным строкам или столбцам
### Когда и как применять
Его применяют, когда диагностировали переобучение, то есть сеть отлично работает с тренировочными данными и плохо с тестовыми.<br>
Dropout чаще всего применяют к полносвязным слоям сети. Его используют и со свёрточными и рекуррентными слоями, однако это далеко не всегда оправдано.<br>
Также довольно редко применяют BatchNorm и Dropout вместе. Как правило, BatchNorm используют чаще. <br>
### Model.eval() метод
В Pytorch у модели есть два метода: `train()` и `eval()`. Один метод используется для фазы обучения, другой для фазы предсказания. Но что именно делает `eval()`? Он меняет поведение слоёв `BatchNorm` и `Dropout`. При `eval()` в `BatchNorm` статистики батча не считаются, а используются аккумулированные и подсчитанные ранее. А `Dropout` слой перестаёт работать, то есть нейроны не «выключаются» ($p=0$). 

$\sum_{i=0}^n \pi^2 = \Delta\frac{(n^2+\epsilon)(2n+1)}{6\phi}$<br>
$\sum_{i=0}^n \pi^2 = \frac{(n^2+\epsilon)(2n+1)}{6\phi}$<br>